In [1]:
import os

from joblib import Parallel, delayed
from tqdm.auto import tqdm
from tqdm import tqdm

In [2]:
esm_data_dir = "/scratch/09101/whatever/data/esmfold_atlas"
esm_data_dir_raw = "/scratch/00946/zzhang/data/esmfold/"

In [7]:
def get_size_and_diff(i):
    files = list(map(lambda x: x[:-4], os.listdir(f"{esm_data_dir}/{i:03d}")))
    files_raw = list(map(lambda x: x[:-4], os.listdir(f"{esm_data_dir_raw}/{i:03d}")))
    lacked_files = tuple(set(files_raw) - set(files))
    size = sum([os.path.getsize(f"{esm_data_dir}/{i:03d}/{f}.npz") for f in files]) / 1_000_000
    size_raw = sum([os.path.getsize(f"{esm_data_dir_raw}/{i:03d}/{f}.pdb") for f in files]) / 1_000_000
    return lacked_files, size, size_raw

In [10]:
lacked_files, sizes, sizes_raw = zip(*Parallel(n_jobs=128)(delayed(get_size_and_diff)(i) for i in tqdm(range(1000))))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:48<00:00,  3.46it/s]


In [15]:
with open("slurm_log/bad_proteins.txt", "w") as f:
    for i in [j for i in lacked_files for j in i]:
        print(i, file=f)

In [16]:
with open("slurm_log/size_stats.txt", "w") as f:
    print(sum(sizes) / 1000, file=f)
    print(sum(sizes_raw) / 1000, file=f)

In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
data_dir = "/scratch/09101/whatever/data/esmfold_atlas"

In [4]:
df_full = pd.read_parquet(f"{data_dir}/stats.parquet")
df_rep = pd.read_parquet(f"{data_dir}/stats_rep.parquet")

In [8]:
df_full.columns

Index(['id', 'ptm', 'plddt', 'num_conf', 'len'], dtype='object')

In [31]:
df_full["perc_conf"] = df_full.num_conf / df_full.len
df_rep["perc_conf"] = df_rep.num_conf / df_rep.len

In [10]:
(df_rep.plddt < 0.7).sum()

2

In [11]:
(df_rep.ptm < 0.7).sum()

8920

In [13]:
bad_pro = pd.read_csv("slurm_log/bad_proteins.txt", header=None)
bad_pro.columns = ["id"]
bad_pro["reason"] = "zero models"


In [21]:
df_low_plddt = df_rep[df_rep.plddt < 0.7]
bad_pro = pd.concat(
    [
        bad_pro,
        pd.DataFrame(dict(id=df_low_plddt.id, reason="low plddt " + df_low_plddt.plddt.map(str)))
    ]
)

In [24]:
df_low_ptm = df_rep[df_rep.ptm < 0.7]
bad_pro = pd.concat(
    [
        bad_pro,
        pd.DataFrame(dict(id=df_low_ptm.id, reason="low ptm " + df_low_ptm.ptm.map(str)))
    ]
)

In [25]:
bad_pro.shape

(10223, 2)

In [26]:
bad_pro.to_csv("slurm_log/bad_proteins.csv", index=False)

In [28]:
df_rep = pd.read_parquet(f"{data_dir}/stats_rep.parquet")
df_rep = df_rep[~df_rep.id.isin(bad_pro.id)]

In [29]:
df_rep.shape

(36977707, 5)

In [54]:
df_rep.to_parquet(f"{data_dir}/stats_rep.parquet")

In [38]:
fig, axs = plt.subplots(4, 1, figsize=(10, 20))
df_rep.ptm.plot(kind="hist", bins=100, ax=axs[0])
df_rep.plddt.plot(kind="hist", bins=100, ax=axs[1])
df_rep.perc_conf.plot(kind="hist", bins=100, ax=axs[2])
df_rep.len.plot(kind="hist", bins=100, ax=axs[3])
axs[0].set_xlabel("pTM")
axs[1].set_xlabel("plDDT")
axs[2].set_xlabel("Ratio of plDDT > 0.7 residues")
axs[3].set_xlabel("Length")
fig.savefig("imgs/esmfold_atlas_dist_rep.jpg", dpi=150)

In [39]:
fig, axs = plt.subplots(4, 1, figsize=(10, 30))
df_full.ptm.plot(kind="hist", bins=100, ax=axs[0])
df_full.plddt.plot(kind="hist", bins=100, ax=axs[1])
df_full.perc_conf.plot(kind="hist", bins=100, ax=axs[2])
df_full.len.plot(kind="hist", bins=100, ax=axs[3])
axs[0].set_xlabel("pTM")
axs[1].set_xlabel("plDDT")
axs[2].set_xlabel("Ratio of plDDT > 0.7 residues")
axs[3].set_xlabel("Length")
fig.savefig("imgs/esmfold_atlas_dist.jpg", dpi=150)

In [43]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.hist2d(x=df_rep.plddt, y=df_rep.ptm, cmap="hot_r", bins=(30, 30))
ax.set_xlabel("plDDT")
ax.set_ylabel("pTM")
fig.savefig("imgs/esmfold_atlas_plddt_ptm_rep.jpg")

In [45]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.hist2d(x=df_rep.plddt, y=df_rep.perc_conf, bins=(30, 60), cmap="hot_r")
ax.set_xlabel("plDDT")
ax.set_ylabel("Ratio of plDDT > 0.7 residues")
fig.savefig("imgs/esmfold_atlas_plddt_conf_rep.jpg")

In [51]:
df_full = df_full.dropna(subset=["plddt", "ptm", "perc_conf"])

In [52]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.hist2d(x=df_full.plddt, y=df_full.ptm, bins=(100, 100), cmap="hot_r")
ax.set_xlabel("plDDT")
ax.set_ylabel("pTM")
ax.axvline(0.7, linestyle='--', color="r")
ax.axhline(0.7, linestyle='--', color="r")
fig.savefig("imgs/esmfold_atlas_plddt_ptm.jpg")

In [53]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.hist2d(x=df_full.plddt, y=df_full.perc_conf, bins=(100, 100), cmap="hot_r")
ax.set_xlabel("plDDT")
ax.set_ylabel("Ratio of plDDT > 0.7 residues")
ax.axvline(0.7, linestyle='--', color="r")
fig.savefig("imgs/esmfold_atlas_plddt_conf.jpg")

In [61]:
from tqdm.auto import tqdm

In [62]:
id_to_dir = {
    j: i
    for i in tqdm(os.listdir(data_dir)) if os.path.isdir(f"{data_dir}/{i}")
    for j in os.listdir(f"{data_dir}/{i}")
}

  1%|█▌                                                                                                                                            | 11/1002 [00:20<30:45,  1.86s/it]


  0%|          | 0/1002 [00:00<?, ?it/s]

In [63]:
for i in range(70, 100):
    for j in range(70, 100):
        os.makedirs(f"{data_dir}/{i}_{j}", exist_ok=True)

In [64]:
import shutil

In [67]:
for rep in tqdm(df_rep.itertuples()):
    file_ = f"{rep.id}.npz"
    src = f"{data_dir}/{id_to_dir[file_]}/{file_}"
    dst = f"{data_dir}/{int(rep.ptm * 100)}_{int(rep.plddt * 100)}/{file_}"
    shutil.copy(src, dst)

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [1]:
import os
import shutil

In [2]:
esm_dir = "/scratch/09120/sk844/validation_set_casp15/esmfold_predictions"
for i in os.listdir(esm_dir):
    shutil.copy(f"{esm_dir}/{i}", f"./val_res/casp15_pdb_esm/{i.split()[0]}.pdb")

In [4]:
colab_dir = "/scratch/09120/sk844/validation_set_casp15/colabfold_predictions"
for i in os.listdir(colab_dir):
    if not i.endswith(".pdb"):
        continue
    model_id = i[-5]
    shutil.copy(f"{colab_dir}/{i}", f"./val_res/casp15_pdb_cf{model_id}/{i.split('_')[0]}.pdb")

In [5]:
colab_ss_dir = "/scratch/09120/sk844/validation_set_casp15/colabfold_sseq_predictions"
for i in os.listdir(colab_ss_dir):
    if not i.endswith(".pdb"):
        continue
    model_id = i[-5]
    shutil.copy(f"{colab_ss_dir}/{i}", f"./val_res/casp15_pdb_cf_ss{model_id}/{i.split('_')[0]}.pdb")